## Summary

Calculate features using [AlphaFold](https://github.com/deepmind/alphafold).

```bash
export NOTEBOOK_PATH="$(realpath 31_run_alphafold.ipynb)"
export DATASET_NAME="cagi6-sherloc"
export DATASET_PATH="30_cagi6_sherloc/input-data-gby-protein.parquet"
export ORIGINAL_ARRAY_TASK_COUNT=4182

# === Cedar ===
# p100
# sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-1000 --time 24:00:00 --gres=gpu:p100:1 ../scripts/run_notebook_gpu.sh
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1001-2000 --time 24:00:00 --gres=gpu:p100:1 ../scripts/run_notebook_gpu.sh

# v100l
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=2001-3000 --time 24:00:00 --gres=gpu:v100l:1 ../scripts/run_notebook_gpu.sh
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=3001-3800 --time 24:00:00 --gres=gpu:v100l:1 ../scripts/run_notebook_gpu.sh

# CPU
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=3801-4182 --time 72:00:00 ../scripts/run_notebook_cpu.sh



# === Graham ===
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-3500 --time 24:00:00 --gres=gpu:t4:1 ../scripts/run_notebook_gpu.sh
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=181-200 --time 24:00:00 --gres=gpu:v100:1 ../scripts/run_notebook_gpu.sh
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=201-306 ../scripts/run_notebook_cpu.sh



sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=3,7,9,12,13,14,17,20,25,29,30,31,34,36,37,39 --time 24:00:00 --gres=gpu:v100:1 ../scripts/run_notebook_gpu.sh
```

---

## Imports

In [1]:
import concurrent.futures
import os
import re
import subprocess
import sys
import tempfile
from pathlib import Path

import jax
import kmbio
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from elaspic2.plugins.alphafold import (
    AlphaFold,
    AlphaFoldAnalyzeError,
    AlphaFoldBuildError,
)
from jax.lib import xla_bridge
from kmbio import PDB
from tqdm.notebook import tqdm

## Parameters

In [2]:
NOTEBOOK_DIR = Path("31_run_alphafold_wt").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/scratch/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold_wt')

In [3]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir

print(tempfile.gettempdir())

/tmp


In [4]:
DATASET_NAME = os.getenv("DATASET_NAME")
DATASET_PATH = os.getenv("DATASET_PATH")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv(
    "SLURM_ARRAY_TASK_COUNT"
)

TASK_ID = int(TASK_ID) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

('cagi6-sherloc',
 '30_cagi6_sherloc/input-data-gby-protein.parquet',
 None,
 4182)

In [5]:
DEBUG = TASK_ID is None

if DEBUG:
    DATASET_NAME = "cagi6-sherloc"
    DATASET_PATH = str(
        NOTEBOOK_DIR.parent.joinpath(
            "30_cagi6_sherloc", "input-data-gby-protein.parquet"
        )
    )
    TASK_ID = 2800
    TASK_COUNT = 4182
else:
    assert DATASET_NAME is not None
    assert DATASET_PATH is not None
    DATASET_PATH = Path(DATASET_PATH).expanduser().resolve()
    assert TASK_ID is not None
    assert TASK_COUNT is not None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

('cagi6-sherloc',
 '/scratch/strokach/workspace/elaspic2-cagi6/notebooks/30_cagi6_sherloc/input-data-gby-protein.parquet',
 2800,
 4182)

In [6]:
device = xla_bridge.get_backend().platform

device

'gpu'

In [7]:
output_file = NOTEBOOK_DIR.joinpath(
    DATASET_NAME, f"shard-{TASK_ID}-of-{TASK_COUNT}.parquet"
)
output_file.parent.mkdir(exist_ok=True)

output_file

PosixPath('/scratch/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold_wt/cagi6-sherloc/shard-2800-of-4182.parquet')

In [8]:
if output_file.is_file():
    raise Exception("Already finished!")

## Workspace

### Initialize model

In [9]:
AlphaFold.load_model(device=device)

### Load data

In [10]:
pfile = pq.ParquetFile(DATASET_PATH)

pfile.num_row_groups

4182

In [11]:
assert TASK_COUNT == pfile.num_row_groups

In [12]:
input_df = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [13]:
display(input_df.head(2))
print(len(input_df))

,protein_id,mutation_id,mutation,effect,sequence,structure,alignment
0,Q9NZC9,"[NM_014140.3:c.1594C>T, NM_014140.3:c.1762G>T,...","[L532F, A588S, S579L, R644W, R645C, V650I, R65...","[Uncertain significance, Uncertain significanc...",MSLPLTEEQRKKIEENRQKALARRAEKLLAEQHQRTSSGTSIAGNP...,HEADER ...,"[>101\n, MSLPLTEEQRKKIEENRQKALARRAEKLLAEQHQRTS..."


1


In [14]:
tup = next(input_df.itertuples(index=False))

iterable_fields = []
for field in tup._fields:
    try:
        if len(getattr(tup, field)) == len(tup.mutation):
            iterable_fields.append(field)
    except TypeError:
        pass

iterable_fields

['mutation_id', 'mutation', 'effect']

In [15]:
if DEBUG:
    for field in iterable_fields:
        input_df[field] = input_df[field].str[:3]

### Score mutations

In [16]:
def validate_mutation(mutation):
    aa = "GVALICMFWPDESTYQNKRH"
    if re.search(f"^[{aa}][1-9]+[0-9]*[{aa}]$", mutation) is None:
        print(f"Skipping mutation {mutation} because it appears to be malformed.")
        return False

    if mutation[0] == mutation[-1]:
        print(
            f"Skipping mutation {mutation} because the wildtype and mutant residues are the same."
        )
        return False

    return True

In [17]:
def predictions_to_embeddings(predictions):
    return {
        "plddt": predictions["plddt"],
        "max_predicted_aligned_error": predictions["max_predicted_aligned_error"].item(),
        "ptm": predictions["ptm"].item(),
        #
        "experimentally_resolved": predictions["experimentally_resolved"]["logits"].to_py().tolist(),
        "predicted_lddt": predictions["predicted_lddt"]["logits"].to_py().tolist(),
        #
        "msa_first_row": predictions["representations"]["msa_first_row"].to_py().tolist(),
        "single": predictions["representations"]["single"].to_py().tolist(),
        "structure_module": predictions["representations"]["structure_module"].to_py().tolist(),
    }

In [18]:
results = []
for tup in tqdm(input_df.itertuples(index=False), total=len(input_df)):
    assert all(
        [(len(getattr(tup, field)) == len(tup.mutation)) for field in iterable_fields]
    )

    data = AlphaFold.build(tup.sequence, ligand_sequence=None, msa=tup.alignment)

    results.append(
        {
            "protein_id": tup.protein_id,
        }
        | predictions_to_embeddings(data.predictions)
    )

results_df = pd.DataFrame(results)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp1o9w91q1.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp1o9w91q1.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp1o9w91q1.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpkxxp5bgt.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpkxxp5bgt.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpkxxp5bgt.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



In [19]:
display(results_df.head(2))
print(len(results_df))

,protein_id,plddt,max_predicted_aligned_error,ptm,experimentally_resolved,predicted_lddt,msa_first_row,single,structure_module
0,Q9NZC9,"[31.192487998268913, 27.489300698296564, 30.61...",31.75,0.586611,"[[0.014223925769329071, 0.11759003251791, 0.08...","[[-3.9497342109680176, -3.859968423843384, -2....","[[-1.3386276960372925, 2.880795955657959, 2.09...","[[-6.072865009307861, 70.27467346191406, 9.798...","[[0.0007754936814308167, -0.005730725824832916..."


1


### Save results

In [20]:
if not DEBUG:
    pq.write_table(pa.Table.from_pandas(results_df, preserve_index=False), output_file)